# Advanced `if / elif / else`

Beyond the basics: readability patterns, guard clauses, conditional expressions, assignment expressions (`:=`), truthiness pitfalls, `any`/`all`, and when *not* to use `if` (EAFP vs LBYL).

## 1) `if / elif / else`: mutually exclusive branches
Use `elif` (not nested `if`) when branches are **mutually exclusive**. It makes intent and performance clearer.

In [1]:
score = 72
if score >= 90:
    grade = 'A'
elif score >= 80:
    grade = 'B'
elif score >= 70:
    grade = 'C'
elif score >= 60:
    grade = 'D'
else:
    grade = 'F'
print(grade)  # C

C


### Tip: Order from most-restrictive to least-restrictive
Top checks should be the *strongest* (e.g., `>= 90`) so later checks can assume earlier ones failed.

## 2) Guard clauses: reduce nesting, improve readability
When a function has preconditions, **return early** on invalid states. This flattens code and avoids deep nesting.

In [2]:
def withdraw(balance: float, amount: float, enabled: bool) -> float:
    if not enabled:
        print("Account disabled")
        return balance
    if amount <= 0:
        print("Invalid amount")
        return balance
    if amount > balance:
        print("Insufficient funds")
        return balance
    # Happy path below
    print("Withdrawal approved")
    return balance - amount

assert withdraw(1000, 50, True) == 950
assert withdraw(1000, -1, True) == 1000
assert withdraw(1000, 5000, True) == 1000
assert withdraw(1000, 50, False) == 1000

Withdrawal approved
Invalid amount
Insufficient funds
Account disabled


## 3) Conditional expression (`x if cond else y`)
Use for short, expression-level choices. Keep it simple—avoid complex nesting.

Good:
```python
status = "hot" if temp > 30 else "ok"
```
Be cautious when `0`/`""` are valid values—don’t use `or` as a drop-in replacement.

In [3]:
temp = 31
status = "hot" if temp > 30 else "ok"
assert status == "hot"

# Nested, but still readable (limit to 2 levels max in practice)
def bucketize(n: int) -> str:
    return ("negative" if n < 0 else ("zero" if n == 0 else "positive"))
assert bucketize(-1) == "negative"
assert bucketize(0) == "zero"
assert bucketize(5) == "positive"

## 4) Assignment expression (`:=`, the walrus) in `if`
Use `:=` to reuse a computed value inside the condition and body **without** recomputing it. Great for input parsing or expensive calls.

Guideline: Only when it enhances clarity—don’t cram logic into the condition.

In [4]:
def read_first_nonempty(lines):
    for raw in lines:
        if (s := raw.strip()):  # truthy if not empty after strip
            return s
    return None

assert read_first_nonempty(["  ", "\t", " hello "]) == "hello"
assert read_first_nonempty(["  ", "\t"]) is None

## 5) Truthiness pitfalls
- `0`, `0.0`, `''`, `[]`, `{}`, `set()`, `None` are **falsy**.
- Distinguish `0` (valid value) from `None` (missing). Prefer `is None`/`is not None` for sentinel checks.
- Avoid `if x:` when `x` can be `0` but is valid; be explicit.

In [5]:
count = 0            # valid value!
limit = None         # missing

msg_count = ("no data" if count is None else f"count={count}")
msg_limit = ("no limit" if limit is None else f"limit={limit}")
assert msg_count == "count=0"
assert msg_limit == "no limit"

## 6) `any`/`all` in conditionals
These short-circuit internally and keep complex checks tidy.
- `any(iterable)` → True if **any** item truthy
- `all(iterable)` → True if **all** items truthy (True for empty iterables — vacuous truth)

In [6]:
user = {"email": "a@b.com", "phone": "", "backup": None}
if any(user.get(k) for k in ("email", "phone", "backup")):
    contactable = True
else:
    contactable = False
assert contactable is True

password = "Abc123!"  # demo only
rules = [
    any(c.islower() for c in password),
    any(c.isupper() for c in password),
    any(c.isdigit() for c in password),
    any(not c.isalnum() for c in password),
    len(password) >= 8,
]
assert all(rules) is False  # length rule fails

## 7) LBYL vs EAFP (When *not* to use `if`)
- **LBYL** (Look Before You Leap): check conditions **before** acting (often via `if`).
- **EAFP** (Easier to Ask Forgiveness than Permission): try the action, handle exceptions.

Prefer EAFP when it removes race conditions and avoids duplicate work. Use `if` (LBYL) for cheap checks or to avoid exceptions in hot paths.

In [7]:
def get_int_safe(d, key):
    # LBYL
    if key in d and isinstance(d[key], int):
        return d[key]
    return 0

def get_int_eafp(d, key):
    # EAFP
    try:
        v = d[key]
        return v if isinstance(v, int) else 0
    except KeyError:
        return 0

sample = {"a": 10}
assert get_int_safe(sample, "a") == 10
assert get_int_safe(sample, "b") == 0
assert get_int_eafp(sample, "a") == 10
assert get_int_eafp(sample, "b") == 0

## 8) Combining comparisons & guarding expensive work
Use chained comparisons; pre-check cheap conditions before expensive calls.

In [8]:
def expensive(x):
    # simulate cost
    for _ in range(10000):
        pass
    return x * x

x = 10
if 0 < x < 20 and (y := expensive(x)) > 50:
    result = y
else:
    result = None
assert result == 100

## 9) Pattern: mapping ranges to labels using `if` vs data-driven
Advanced note: for many disjoint numeric ranges, `if/elif` is fine. For lots of categories, consider data-driven tables/functions. Still, here’s a clean `if/elif` range mapper:

In [9]:
def letter_grade(score: int) -> str:
    if score >= 90: return 'A'
    if score >= 80: return 'B'
    if score >= 70: return 'C'
    if score >= 60: return 'D'
    return 'F'

assert letter_grade(95) == 'A'
assert letter_grade(72) == 'C'
assert letter_grade(12) == 'F'

---
## 10) Exercises
Try these first; solutions follow.

**Exercise A — Normalized division**

Write `safe_ratio(a, b)` that returns:
- `None` if `b == 0`,
- otherwise `a / b`.
Use a **guard clause** to avoid division by zero. Don’t use `try/except` here.

In [10]:
def safe_ratio(a, b):
    # TODO: implement with a guard clause
    raise NotImplementedError()

# Quick checks (uncomment when implemented)
# assert safe_ratio(10, 2) == 5
# assert safe_ratio(1, 0) is None

**Exercise B — Choose display name**

Write `display_name(primary, fallback)` returning:
- `primary` if it is **not None** (even if empty string),
- else `fallback`.
Use explicit `is not None` rather than `or`.

In [11]:
def display_name(primary, fallback):
    # TODO: keep empty string if primary is ''
    raise NotImplementedError()

# assert display_name("Alice", "<anon>") == "Alice"
# assert display_name("", "<anon>") == ""
# assert display_name(None, "<anon>") == "<anon>"

**Exercise C — First valid token**

Given a list of strings, return the first token that is **non-empty after stripping**, otherwise `None`.
Use the walrus operator inside the `if` condition.

In [12]:
def first_token(tokens):
    # TODO: use (t := tok.strip()) in the conditional
    raise NotImplementedError()

# assert first_token(["  ", " x ", "y"]) == "x"
# assert first_token(["  ", "\t"]) is None

**Exercise D — Range label with `elif`**

Write `temperature_label(c)` that returns one of: `"freezing"` (<= 0), `"cold"` (1–10), `"mild"` (11–20), `"warm"` (21–30), `"hot"` (> 30). Use `if/elif/else` (mutually exclusive).

In [13]:
def temperature_label(c: int) -> str:
    # TODO
    raise NotImplementedError()

# assert temperature_label(-3) == "freezing"
# assert temperature_label(10) == "cold"
# assert temperature_label(15) == "mild"
# assert temperature_label(25) == "warm"
# assert temperature_label(31) == "hot"

---
## Solutions (reveal when ready)

In [14]:
# Exercise A
def safe_ratio(a, b):
    if b == 0:
        return None
    return a / b

# Exercise B
def display_name(primary, fallback):
    return primary if primary is not None else fallback

# Exercise C
def first_token(tokens):
    for tok in tokens:
        if (t := tok.strip()):
            return t
    return None

# Exercise D
def temperature_label(c: int) -> str:
    if c <= 0:
        return "freezing"
    elif c <= 10:
        return "cold"
    elif c <= 20:
        return "mild"
    elif c <= 30:
        return "warm"
    else:
        return "hot"

# quick self-checks
assert safe_ratio(10, 2) == 5
assert safe_ratio(1, 0) is None
assert display_name("Alice", "<anon>") == "Alice"
assert display_name("", "<anon>") == ""
assert display_name(None, "<anon>") == "<anon>"
assert first_token(["  ", " x ", "y"]) == "x"
assert first_token(["  ", "\t"]) is None
assert temperature_label(-3) == "freezing"
assert temperature_label(10) == "cold"
assert temperature_label(15) == "mild"
assert temperature_label(25) == "warm"
assert temperature_label(31) == "hot"

---
## Final Tests
Run to validate all examples and solutions.

In [15]:
# Re-run a subset plus extra edges
assert (lambda s: ("hot" if s > 30 else "ok"))(31) == "hot"
assert read_first_nonempty(["\n", "  hi  "]) == "hi"
assert (lambda lines: read_first_nonempty(lines))(["  "]) is None

for v in [0, 0.0, "", [], {}, set(), None]:
    assert (bool(v) is False) == (v in (0, 0.0, "", [], {}, set(), None))

password = "Abc123!Z"
rules = [
    any(c.islower() for c in password),
    any(c.isupper() for c in password),
    any(c.isdigit() for c in password),
    any(not c.isalnum() for c in password),
    len(password) >= 8,
]
assert all(rules)

assert safe_ratio(0, 5) == 0
assert display_name("", "fallback") == ""
assert first_token(["  a  "]) == "a"
assert temperature_label(0) == "freezing"

print("All advanced if/else checks passed ✅")

All advanced if/else checks passed ✅
